In [2]:
import pandas as pd
import numpy as np

root = "/home/austin/Github/kaggle-ncaa-2018/"

# reg_season_detailed = pd.read_csv(root + "original_data/RegularSeasonDetailedResults.csv")
# tourney_detailed = pd.read_csv(root + "original_data/NCAATourneyDetailedResults.csv")
# reg_season_composite = pd.read_csv(root + "original_data/RegularSeasonCompositeStats.csv")
# seeds = pd.read_csv(root + "original_data/NCAATourneySeeds.csv")
# elo = pd.read_csv(root + "derived_data/season_elos.csv")

seasons = range(2003, 2018)

In [3]:
# Make & start filling initial dataframe
all_data = pd.DataFrame(columns=["Season", "TeamID"])
index = 0
for i in range(len(seasons)):
    season = seasons[i]
    team_ids_for_season = pd.unique(tourney_detailed.loc[tourney_detailed["Season"] == season, ["WTeamID", "LTeamID"]].values.ravel('K'))
    for j in range(len(team_ids_for_season)):
        all_data.loc[index, "Season"] = season
        all_data.loc[index, "TeamID"] = team_ids_for_season[j]
        index += 1

KeyError: "None of [['WTeamID', 'LTeamID']] are in the [columns]"

In [3]:
# Add regular season stats

# Relevant stats
stats_columns = ["FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]
w_team_stats = ["WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]
l_team_stats = ["LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]

# Add empty columns for new stats being added
all_data[stats_columns] = pd.DataFrame(index=range(all_data.shape[0]), columns=stats_columns)

# Loop through seasons
for season in seasons:
    games_for_season = reg_season_detailed.loc[reg_season_detailed["Season"] == season]
    team_ids_for_season = all_data.loc[all_data["Season"] == season, "TeamID"].values.ravel()

    # Loop through team IDs in the season
    for team_id in team_ids_for_season:
        games = games_for_season.loc[(games_for_season["WTeamID"] == team_id) | (games_for_season["LTeamID"] == team_id)].reset_index()
        stats = pd.DataFrame(index=range(games.shape[0]), columns=stats_columns)
        for i, row in games.iterrows():
            if row["WTeamID"] == team_id:
                stats.loc[i, stats_columns] = row[w_team_stats].as_matrix()
            else:
                stats.loc[i, stats_columns] = row[l_team_stats].as_matrix()
        
        all_data.loc[(all_data["Season"] == season) & (all_data["TeamID"] == team_id), stats_columns] = stats.mean(axis=0).ravel()

In [4]:
# Add composite stats, seeds, and elo

comp_columns = ["PIE", "FG_PCT", "TURNOVER_RATE", "OFF_REB_PCT", "FT_RATE", "4FACTOR", "OFF_EFF", "DEF_EFF",
                "ASSIST_RATIO", "DEF_REB_PCT", "FT_PCT", "WINPCT"]

# Add empty columns for new stats being added
all_data[comp_columns + ["Seed", "Elo"]] = pd.DataFrame(index=range(all_data.shape[0]), columns=comp_columns + ["Seed", "Elo"])

# Loop through all season/team pairs
for i, row in all_data.iterrows():
    # Get season and team ID for the row
    season = row["Season"]
    team_id = row["TeamID"]
    
    # Get composite stats
    season_team_data = reg_season_composite.loc[(reg_season_composite["Season"] == season) & (reg_season_composite["TeamID"] == team_id)]
    
    # Add composite stats
    all_data.loc[i, comp_columns] = season_team_data[comp_columns].as_matrix().ravel()
    
    # Add seed
    seed = seeds.loc[(seeds["Season"] == season) & (seeds["TeamID"] == team_id)]["Seed"].reset_index(drop=True)[0]
    all_data.loc[i, "Seed"] = int(seed[1:3])
    
    # Add elo
    all_data.loc[i, "Elo"] = elo.loc[(elo["season"] == season) & (elo["team_id"] == team_id)]["season_elo"].reset_index(drop=True)[0]

In [5]:
# all_data.to_csv(root + "derived_data/Master.csv")

In [52]:
master = pd.read_csv(root + "derived_data/Master.csv")
teams = pd.read_csv("/home/austin/Github/DataFiles/TeamSpellings.csv", encoding='latin1')

In [53]:
# Add sports ref data
sportsref_cols = ["Pace", "ORtg", "FTr", "3PAr", "TS%", "TRB%", "AST%", "STL%", "BLK%", "eFG%", "TOV%", "ORB%", "FT/FGA"]
master[sportsref_cols] = pd.DataFrame(index=range(master.shape[0]), columns=sportsref_cols)

In [54]:
for season in range(2010,2018):
    print(season)
    season_data = pd.read_csv("/home/austin/Github/sportsref/" + str(season) + ".csv")
    
    # Get all teams and make them lower case
    season_data_teams = season_data.loc[:, "School"].ravel()
    for i in range(len(season_data_teams)):
        season_data_teams[i] = season_data_teams[i].lower()
    season_data.loc[:, "School"] = season_data_teams
#     print(season_data_teams)
    
    # Get team IDs for season tourney
    team_ids_for_season = pd.unique(master.loc[master["Season"] == season, ["TeamID"]].values.ravel('K'))
    missed = team_ids_for_season
    for team_id in team_ids_for_season:
        team_names = teams.loc[teams["TeamID"] == team_id, "TeamNameSpelling"].tolist()
        for team_name in team_names:
#             print(team_name)
            if team_name in season_data_teams:
                missed = np.delete(missed, np.argwhere(missed==team_id))
                team_data = season_data.loc[season_data["School"] == team_name, sportsref_cols].as_matrix().ravel()
                master.loc[(master["TeamID"] == team_id) & (master["Season"] == season), sportsref_cols] = team_data
                
    print(missed)

2010
[]
2011
[]
2012
[]
2013
[]
2014
[]
2015
[]
2016
[]
2017
[]


In [55]:
print(master.loc[:,"FTr"])

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
       ...  
966    0.381
967    0.364
968     0.37
969    0.432
970    0.329
971    0.404
972    0.413
973    0.396
974    0.348
975    0.242
976    0.341
977    0.364
978    0.317
979    0.372
980    0.409
981    0.398
982    0.376
983    0.431
984    0.393
985    0.363
986    0.346
987    0.383
988    0.343
989    0.443
990     0.31
991    0.277
992     0.31
993    0.328
994     0.36
995    0.436
Name: FTr, Length: 996, dtype: object


In [57]:
master.to_csv("/home/austin/Github/kaggle-ncaa-2018/derived_data/Master.csv")